### Pipeline that Generates key based on Local Time and Fibonacci Sequences

In [1]:
import kfp
import typing

#### Token Generator

In [2]:
from typing import NamedTuple
from kfp.components import *

def token_generator() -> str:
    from datetime import datetime
    now = datetime.now()
    token = now.strftime("%H%M")
    while(len(token)<4):
        token = str(0)+str(token)
    return token

In [3]:
kfp_token_generator = kfp.components.func_to_container_op(func = token_generator, 
                                                          output_component_file = './tok_gen.yaml',
                                                          packages_to_install = ['datetime'])


#### Unique Key Generated based on the Token Passed

In [4]:
def key_gen(tok : str)-> NamedTuple('S_outs', [('outkey', int), ('token', str)]):
    
    tok = str(tok)
    
    def fib_generate(a,s):
        p = 0
        q = s
        r = s
        for x in range(a-1):
            r = p+q
            p = q
            q = r
        return r
    
    out_key = fib_generate(int(str(tok[1])+str(tok[3])),int(tok[0])+1)
    print("Generated key {}".format(out_key))                        
    return (out_key, tok)

#### Key Decoder - Decoded key matches with Passed key only if Token is Same

In [5]:
def key_decode(tok: str, seq_two: int)-> str:
    
    def fib_generate(a,s):
        p = 0
        q = s
        r = s
        for x in range(a-1):
            r = p+q
            p = q
            q = r
        return r
    
    def fib_decode(f,s):
        x = 0
        y = fib_generate(0,s)
        while(f>=fib_generate(x,s)):
            y = fib_generate(x,s)
            x+=1
        return y

    if(fib_decode(seq_two, int(tok[0])+1)==seq_two):
        print("Keys match")
        out_str = "Keys Match"
    else:
        print("Keys don't match")
        out_str = "Keys Don't Match"
                                                                  
    return out_str

In [6]:
kfp_key_gen = kfp.components.func_to_container_op(func = key_gen, 
                                                          output_component_file = './kgen.yaml')


In [7]:
kfp_key_decode = kfp.components.func_to_container_op(func = key_decode, 
                                                          output_component_file = './kdecode.yaml')


#### Token Generator 2

In [8]:
def token_generator_two() -> str:
    from datetime import datetime
    now = datetime.now()
    token = now.strftime("%H%M")
    token = int(token)+7
    while(len(str(token))<4):
        token = str(0)+str(token)
    return str(token)

In [9]:
kfp_token_generator_two = kfp.components.func_to_container_op(func = token_generator_two, 
                                                          output_component_file = './tok_gen_two.yaml',
                                                          packages_to_install = ['datetime'])


#### Defining the Pipeline Execution Sequence and Input-Output scheme

In [10]:
import kfp.dsl as dsl

@dsl.pipeline(name='Fibonacci Security Key Generator',description='Generates automatically a random key from a token based on Fibonacci sequences')
def Keygen_func():
    #Passing pipeline parameter and a constant value as operation arguments
    tg_one_task = kfp_token_generator() 
    kg_one_task = kfp_key_gen(tg_one_task.output)
    kg_two_task = kfp_key_gen(tg_one_task.output)
    tg_two_task = kfp_token_generator_two()
    kg_three_task = kfp_key_gen(tg_two_task.output)
    kd_one_task = kfp_key_decode(kg_one_task.outputs['token'], kg_two_task.outputs['outkey'])
    kd_two_task = kfp_key_decode(kg_two_task.outputs['token'], kg_three_task.outputs['outkey'])
    kd_three_task = kfp_key_decode(kg_three_task.outputs['token'], kg_two_task.outputs['outkey'])
    
#For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
#For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax

#### Compiling the Pipeline

In [11]:
pipeline_func = Keygen_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)